In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## GPU Info & Mounting

In [ ]:
## GPU info

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
## Mount Drive

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')
# root_path = 'drive/My Colab Notebooks/'  #change dir to your project folder

Mounted at /content/drive


## Install & Imports

In [ ]:
import os
home = "/content"
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
!pip install torch==1.6.0 torchvision==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

### Install MMF from Source

In [ ]:
!git clone --branch no_feats --config core.symlinks=true https://github.com/rizavelioglu/mmf.git

fatal: destination path 'mmf' already exists and is not an empty directory.


In [ ]:
import os
os.chdir(os.path.join(home, "mmf"))

In [ ]:
!pip install --editable .

## Convert to MMF format

In [ ]:
zip_file_path="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/data/hateful_meme_data.zip"

In [ ]:
!mmf_convert_hm --zip_file=$zip_file_path --password="" --bypass_checksum 1

2021-07-23 07:07:32.837643: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Data folder is /root/.cache/torch/mmf/data
Zip path is /content/drive/MyDrive/Colab Notebooks/hateful_memes/data/hateful_meme_data.zip
Copying /content/drive/MyDrive/Colab Notebooks/hateful_memes/data/hateful_meme_data.zip
Unzipping /content/drive/MyDrive/Colab Notebooks/hateful_memes/data/hateful_meme_data.zip
Extracting the zip can take time. Sit back and relax.
Moving train.jsonl
Moving dev_seen.jsonl
Moving test_seen.jsonl
Moving dev_unseen.jsonl
Moving test_unseen.jsonl
Moving img


In [ ]:
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ | wc -l

ls: cannot access '/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/': No such file or directory
0


## Feature Extraction

### VQA Mask-RCNN

In [ ]:
import os
os.chdir(home)
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark

In [ ]:
!pip install ninja yacs cython matplotlib

In [ ]:
os.chdir(os.path.join(home, "vqa-maskrcnn-benchmark"))
!rm -rf build
!python setup.py build develop

### Extract

In [ ]:
os.chdir(os.path.join(home, "mmf/tools/scripts/features/"))
out_folder = os.path.join(home, "features/")

!python extract_features_vmb.py --config_file "https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model_x152.yaml" \
                                --model_name "X-152" \
                                --output_folder $out_folder \
                                --image_dir "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/" \
                                --num_features 100 \

In [ ]:
os.chdir(home)
# !zip -r "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features.zip" features

## Fine-tuning w/ VisualBERT

In [ ]:
# os.chdir("/content/drive/MyDrive/Colab Notebooks/hateful_memes/")
# os.getcwd()
# # !unzip "features.zip"

'/content/drive/MyDrive/Colab Notebooks/hateful_memes/features'

In [ ]:
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
log_dir ="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/"
save_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/submissions/"
# Define where image features are
feats_dir = "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features/"
train_dir = "hateful_memes/defaults/annotations/train.jsonl"


In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/fit

In [ ]:

# # Define where train.jsonl is
# train_dir = os.path.join(home, "train_v9.jsonl")
!mmf_run \
        config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
        model="visual_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        checkpoint.max_to_keep=1 \
        checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
        training.tensorboard=True \
        training.checkpoint_interval=50 \
        training.evaluation_interval=50 \
        training.max_updates=3000 \
        training.log_interval=100 \
        dataset_config.hateful_memes.max_features=100 \
        dataset_config.hateful_memes.annotations.train[0]=$train_dir \
        dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
        dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
        dataset_config.hateful_memes.features.train[0]=$feats_dir \
        dataset_config.hateful_memes.features.val[0]=$feats_dir \
        dataset_config.hateful_memes.features.test[0]=$feats_dir \
        training.lr_ratio=0.3 \
        training.use_warmup=True \
        training.batch_size=32 \
        optimizer.params.lr=5.0e-05 \
        env.save_dir=$save_dir \
        env.tensorboard_logdir=$log_dir \